# [MongoDB](https://www.mongodb.com/)

# Install MongoDB

In [1]:
!apt install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libyaml-cpp0.5v5 amd64 0.5.2-4ubuntu1 [150 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mongo-tools amd64 3.6.3-0ubunt

## Start MongoDB server

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


## Connect to MongoDB

In [3]:
from pymongo import MongoClient

client = MongoClient(host='localhost', port= 27017)

In [4]:
client.stats

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stats')

In [5]:
client.list_database_names()

['admin', 'local']

## Shut down Mongo DB server

In [6]:
!service mongodb stop

 * Stopping database mongodb
   ...done.


# Database Management

In [7]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [8]:
from pymongo import MongoClient

client = MongoClient(host='localhost', port= 27017)

In [9]:
client.list_database_names()

['admin', 'local']

## Create Database

In [10]:
db = client.eurostat

In [11]:
client.list_database_names()

['admin', 'local']

In [12]:
client.drop_database('eurostat')

## Create Collection

In [13]:
db = client.eurostat

In [14]:
db.create_collection('directors')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'eurostat'), 'directors')

In [15]:
client.list_database_names()

['admin', 'eurostat', 'local']

In [16]:
list(db.list_collections())

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'eurostat.directors',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('d7660dd3-4f0a-4ace-b735-a4f40d18fe12')},
  'name': 'directors',
  'options': {},
  'type': 'collection'}]

In [17]:
db.drop_collection('directors')

{'nIndexesWas': 1, 'ns': 'eurostat.directors', 'ok': 1.0}

In [18]:
list(db.list_collections())

[]

In [19]:
client.list_database_names()

['admin', 'local']

# CRUD - one

In [20]:
db = client.eurostat
collection = db.directors

collection.full_name

'eurostat.directors'

## Create

In [21]:
res = collection.insert_one({'name': 'Mariana Kotzeva', 
                             'country': 'Bulgaria',
                             'start': 2017})

In [22]:
res.acknowledged

True

In [23]:
res.inserted_id

ObjectId('6184976a906779a5c043aeeb')

In [24]:
collection.insert_one({'name': 'Walter Radermacher', 
                       'country': 'Germany',
                       'start': 2008,
                       'end': 2016})

## Read

In [25]:
list(collection.find())

[{'_id': ObjectId('6184976a906779a5c043aeeb'),
  'country': 'Bulgaria',
  'name': 'Mariana Kotzeva',
  'start': 2017},
 {'_id': ObjectId('6184976a906779a5c043aeec'),
  'country': 'Germany',
  'end': 2016,
  'name': 'Walter Radermacher',
  'start': 2008}]

In [26]:
list(collection.find({'_id': res.inserted_id}))

[{'_id': ObjectId('6184976a906779a5c043aeeb'),
  'country': 'Bulgaria',
  'name': 'Mariana Kotzeva',
  'start': 2017}]

In [27]:
list(collection.find({'country': 'Germany'}))

[{'_id': ObjectId('6184976a906779a5c043aeec'),
  'country': 'Germany',
  'end': 2016,
  'name': 'Walter Radermacher',
  'start': 2008}]

In [28]:
list(collection.find().limit(1))    

[{'_id': ObjectId('6184976a906779a5c043aeeb'),
  'country': 'Bulgaria',
  'name': 'Mariana Kotzeva',
  'start': 2017}]

In [29]:
list(collection.find().skip(1))

[{'_id': ObjectId('6184976a906779a5c043aeec'),
  'country': 'Germany',
  'end': 2016,
  'name': 'Walter Radermacher',
  'start': 2008}]

## Update

In [30]:
res = collection.update_one({'name': 'Mariana Kotzeva'},
                            {'$set': {'end': None}})

In [31]:
res.acknowledged

True

In [32]:
res.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [33]:
list(collection.find())

[{'_id': ObjectId('6184976a906779a5c043aeeb'),
  'country': 'Bulgaria',
  'end': None,
  'name': 'Mariana Kotzeva',
  'start': 2017},
 {'_id': ObjectId('6184976a906779a5c043aeec'),
  'country': 'Germany',
  'end': 2016,
  'name': 'Walter Radermacher',
  'start': 2008}]

## Delete

In [34]:
res = collection.delete_one({'name': 'Mariana Kotzeva'})

In [35]:
res.acknowledged

True

In [36]:
res.deleted_count

1

In [37]:
db.drop_collection('directors')

{'nIndexesWas': 1, 'ns': 'eurostat.directors', 'ok': 1.0}

# CRUD - many

In [38]:
db = client.eurostat
collection = db.directors

collection.full_name

'eurostat.directors'

## Create

In [39]:
directors = list(map(lambda d : {'name': d[0],
                                 'country': d[1],
                                 'start': d[2],
                                 'end': d[3]},
                     [('Rolf Wagenführ', 'West Germany', 1952, 1966),
                      ('Raymond Dumas', 'France', 1966, 1973),
                      ('Jacques Mayer', 'France',	1973, 1977),
                      ('Aage Dornonville de la Cour', 'Denmark', 1977, 1982),
                      ('Pieter de Geus', 'Netherlands', 1982, 1984),
                      ('Silvio Ronchetti', 'Italy',	1984, 1987),
                      ('Yves Franchet', 'France', 1987, 2003),
                      ('Michel Vanden Abeele', 'Belgium', 2003, 2004),
                      ('Günther Hanreich', 'Austria', 2004, 2006),
                      ('Hervé Carré', 'France', 2006, 2008),
                      ('Walter Radermacher', 'Germany', 2008, 2016),
                      ('Mariana Kotzeva', 'Bulgaria', 2017, None)]))

In [40]:
res = collection.insert_many(directors)

In [41]:
res.acknowledged

True

In [42]:
res.inserted_ids

[ObjectId('6184976a906779a5c043aeed'),
 ObjectId('6184976a906779a5c043aeee'),
 ObjectId('6184976a906779a5c043aeef'),
 ObjectId('6184976a906779a5c043aef0'),
 ObjectId('6184976a906779a5c043aef1'),
 ObjectId('6184976a906779a5c043aef2'),
 ObjectId('6184976a906779a5c043aef3'),
 ObjectId('6184976a906779a5c043aef4'),
 ObjectId('6184976a906779a5c043aef5'),
 ObjectId('6184976a906779a5c043aef6'),
 ObjectId('6184976a906779a5c043aef7'),
 ObjectId('6184976a906779a5c043aef8')]

## Read

In [43]:
list(collection.find({'country': 'France'}))

[{'_id': ObjectId('6184976a906779a5c043aeee'),
  'country': 'France',
  'end': 1973,
  'name': 'Raymond Dumas',
  'start': 1966},
 {'_id': ObjectId('6184976a906779a5c043aeef'),
  'country': 'France',
  'end': 1977,
  'name': 'Jacques Mayer',
  'start': 1973},
 {'_id': ObjectId('6184976a906779a5c043aef3'),
  'country': 'France',
  'end': 2003,
  'name': 'Yves Franchet',
  'start': 1987},
 {'_id': ObjectId('6184976a906779a5c043aef6'),
  'country': 'France',
  'end': 2008,
  'name': 'Hervé Carré',
  'start': 2006}]

## Update

In [44]:
res = collection.update_many({'country': 'France' }, {'$set' : { 'city': 'Paris' }} )

In [45]:
res.acknowledged

True

In [46]:
res.modified_count

4

## Delete

In [47]:
res = collection.delete_many({'country' : 'France'})

In [48]:
res.acknowledged

True

In [49]:
collection.count_documents({})

8